In [7]:
import pandas as pd
import numpy as np
import pickle
from multiprocessing import cpu_count
import concurrent.futures
from likelihoods.parth_exact.pyrthenope import Pyrthenope

print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(2/3)

pandas version: 1.3.5
numpy version: 1.20.3


In [8]:
# tau_prior=30
step_size=2
job_name = f"BBN_small"
save_file = f'data_files/{job_name}_s{step_size}.pkl'

use_dali = True
save = True

parth_inout_map = {
    'ETA10': 'eta10',
    'DNNU': 'N_nu',
    'TAU': 'tau'
}

# !!! will need to update card_mod if you change obs
# k, v of intrested obs: expermental errors 
observables = {
    'H2/H': 0.03* 10**(-5),     # https://arxiv.org/abs/1710.11129       
    'Y_p': 0.0026,     # https://arxiv.org/abs/1904.01594 8 26 22
    # 'tau': 0.5          #pdg       
}

# params to vary over and step size, uses parth's card syntax
parameters = {
    'ETA10': step_size,
    'DNNU':  step_size, 
    'TAU':   step_size
}

# cleans up output
obs_and_params = list(observables.keys()) + [parth_inout_map.get(k, k) for k in parameters.keys()]

# these are to match lesnpower output so we can just use the cobaya dali program
lo_spec_id = 'unlensed'
lo_experment = 0 

print(f'Running with {cpu_count()} cores')

Running with 28 cores


In [9]:
def _run(param, mod=None):
    nope = Pyrthenope(card_mod=mod)
    print(f'Running with {param}')
    for (k, v) in param:
        print(f'\tmodifying {k} by {v}, from {nope.card[k]} to {nope.card[k] + v}')
        nope.card[k] += v
    return nope.run()    

def runner(obs, params, cardMods=None, numThreads=cpu_count()):
    # This generates all of our runs, these will run automagically and will only halt when a corresponding .result() is called
    processes = np.empty((2, len(params), len(params), 4), dtype=object)
    with concurrent.futures.ThreadPoolExecutor(max_workers=numThreads) as exec:
        for i, (param1, step1) in enumerate(params.items()):
            processes[0, i, 0, 0] = exec.submit(_run, [(param1, -step1/2)], cardMods)
            processes[0, i, 0, 1] = exec.submit(_run, [(param1,  step1/2)], cardMods)
            for j, (param2, step2) in enumerate(params.items()):
                processes[1, i, j, 0] = exec.submit(_run, [(param1, -step1/2), (param2, -step2/2)], cardMods)
                processes[1, i, j, 1] = exec.submit(_run, [(param1,  step1/2), (param2, -step2/2)], cardMods)
                processes[1, i, j, 2] = exec.submit(_run, [(param1, -step1/2), (param2,  step2/2)], cardMods)
                processes[1, i, j, 3] = exec.submit(_run, [(param1,  step1/2), (param2,  step2/2)], cardMods)

    # derivative vectors
    d1Vec = pd.DataFrame()
    d2Vec = pd.DataFrame()

    #calculate the derivatives
    for i, (param1, step1) in enumerate(params.items()):
        v01 = processes[0, i, 0, 0].result()
        v02 = processes[0, i, 0, 1].result()
        
        d1 = pd.DataFrame((v02[obs] - v01[obs]) / step1).rename(index={0:param1})    
        d1Vec = pd.concat([d1Vec, d1])
        
        print('===============================')
        print(f'fisher step {i} {param1} {step1}')
        print('===============================')
        print(f'd1: \n{d1}\n')
        print(f'v01: \n{v01[obs]} \nv02: \n{v02[obs]}\n')

        for j, (param2, step2) in enumerate(params.items()):
            v11 = processes[1, i, j, 0].result()
            v12 = processes[1, i, j, 1].result()
            v13 = processes[1, i, j, 2].result()
            v14 = processes[1, i, j, 3].result()

            d11 = (v12[obs] - v11[obs]) / step1
            d12 = (v14[obs] - v13[obs]) / step1
            d2 = pd.DataFrame((d12 - d11) / step2).rename(index={0:(param1, param2)})
            d2Vec = pd.concat([d2Vec, d2])
            
            print('===============================')
            print(f'dali step {j} {param2} {step2}')
            print('===============================')
            print(f'd2: \n{d2}\n')
            print(f'v11: \n{v11[obs]} \nv12: \n{v12[obs]} \nv13: \n{v13[obs]} \nv14: \n{v14[obs]}\n')
            print(f'd11: \n{d11} \nd12: \n{d12} \nd2: \n{d2}\n')
        print('=============================== END STEP ===============================')
    
    print('=============================== END RUN ===============================')

    # Finds the (gaussian) 1/sigma^2 errors
    errors = np.array([list(obs.values())])
    isigma2 = pd.DataFrame(
        np.linalg.inv(errors.T * np.identity(len(list(obs))) * errors), 
        index=[list(obs.keys())], columns=[list(obs.keys())])
    d2Ten = d2Vec.values.reshape(len(params), len(params), len(obs))
    
    # calculate fisher, dali3 and dali4
    # https://arxiv.org/pdf/1401.6892.pdf 15
    fisher = np.einsum('ia,ab,jb', d1Vec, isigma2, d1Vec)
    dali3 = np.einsum('ija,ab,kb', d2Ten, isigma2, d1Vec)
    dali4 = np.einsum('ija,ab,klb', d2Ten, isigma2, d2Ten)
    
    # ofisher = np.einsum('ij,jk,wk', d1Vec, isigma2, d1Vec)
    # odali3 = np.einsum('ijk,kw,xw', d2Ten, isigma2, d1Vec)
    # odali4 = np.einsum('ijk,kw,xyw', d2Ten, isigma2, d2Ten)
    
    # print('=============================== END ===============================')
    # print('=============================== END ===============================')
    # print('=============================== END ===============================')
    # print('=============================== END ===============================')
    # print('=============================== END ===============================')
    # print('=============================== END ===============================')
    
    # print(ofisher - fisher)
    # print(odali3 - dali3)
    # print (odali4 - dali4)
    
    print('=============================== DATA ===============================')
    print(f'errors: \n{errors}\n')
    print(f'isigma2: \n{isigma2}\n')
    print(f'd1vec: \n{d1Vec}\n')
    print(f'd2vec: \n{d2Vec}\n')
    print(f'd2Ten: \n{d2Ten}\n')
    print(f'fisher: \n{fisher}\n')
    print(f'dali3: \n{dali3}\n')
    print(f'dali4: \n{dali4}\n')
    print('=============================== DONE ===============================')
    return fisher, dali3, dali4


In [10]:
# baseData = _run([])

print('=============================== INPUT ===============================')
print(f'Obs : Exp Errors   = {observables}')
print(f'Params : Step Size = {parameters}')
print('=============================== RUN ===============================')

fisher, dali3, dali4 = runner(observables, parameters)

# just ensure nonsingualrity
# print(f'sqrt inv fisher: \n{np.sqrt(np.linalg.inv(fisher))}')

=============================== INPUT ===============================
Obs : Exp Errors   = {'H2/H': 3.0000000000000004e-07, 'Y_p': 0.0026}
Params : Step Size = {'ETA10': 2, 'DNNU': 2, 'TAU': 2}
=============================== RUN ===============================
Running with [('ETA10', -1.0)]
	modifying ETA10 by -1.0, from 6.13832 to 5.13832
Running with [('ETA10', 1.0)]
	modifying ETA10 by 1.0, from 6.13832 to 7.13832
Running with [('ETA10', -1.0), ('ETA10', -1.0)]
	modifying ETA10 by -1.0, from 6.13832 to 5.13832
	modifying ETA10 by -1.0, from 5.13832 to 4.13832
Running with [('ETA10', 1.0), ('ETA10', -1.0)]
	modifying ETA10 by 1.0, from 6.13832 to 7.13832
	modifying ETA10 by -1.0, from 7.13832 to 6.13832
Running with [('ETA10', -1.0), ('ETA10', 1.0)]
	modifying ETA10 by -1.0, from 6.13832 to 5.13832
	modifying ETA10 by 1.0, from 5.13832 to 6.13832
Running with [('ETA10', 1.0), ('ETA10', 1.0)]
	modifying ETA10 by 1.0, from 6.13832 to 7.13832
	modifying ETA10 by 1.0, from 7.13832 to 8.

In [11]:
if save:
    fid_values = {'eta10': 6.13832, 'N_nu': 3.0, 'tau': 879.4}
    # fid_values = {
    #     # 'omega_b_h2': baseData['OmegaBh^2'].values[0], #0.02242 # check that this corresponds to our fid eta10 6.13332, is this needed?
    #     'eta10': baseData['eta10'].values[0], #6.13832
    #     'N_nu': baseData['N_nu'].values[0], #3.0,
    #     'tau': baseData['tau'].values[0]
    # }
    saveData = {'cosmoFid': fid_values, 'fisherGaussian': {lo_experment: {lo_spec_id: fisher}}}
    if use_dali:
        saveData['DALI3Gaussian'] =  {lo_experment: {lo_spec_id: dali3}}
        saveData['DALI4Gaussian'] =  {lo_experment: {lo_spec_id: dali4}}

    with open(save_file, 'wb') as file:
        pickle.dump(saveData, file)